# AppointChain: 
### A blockchain based appointment system. For a decentralised and free alternative to individual apps for each host. User friendly so regular / non-tech savvy individuals can use it.

## Import libraries

In [520]:
# Import libraries
import json
import hashlib
import binascii
from datetime import datetime
from urllib.parse import  urlparse

# Cryptography libraries
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

## Build a Wallet

In [521]:
class Wallet(object):
    """
    A wallet is a private/public key pair
    Using RSA digital signature protocol
    """
    def __init__(self):
        random_gen = Crypto.Random.new().read
        self.private_key = RSA.generate(1024, random_gen)
        self.public_key = self.private_key.publickey()
        self.signer = PKCS1_v1_5.new(self.private_key)


    @property
    def address(self):
        """
        Assume public key is address
        """
        # Get address from public key
        address = self.public_key.exportKey(format='DER')
        
        return binascii.hexlify(address).decode('ascii')
    
    
    def sign(self, message):
        """
        Sign a message with this wallet
        """
        message = message if type(message) == str else str(message)
        # Hash message
        h = SHA.new(message.encode('utf8'))
        # Sign encoded message
        signature = self.signer.sign(h)
        
        return binascii.hexlify(signature).decode('ascii')
    
    
    def create_transaction(self, host_address, time_slot, amount):
        """
        Create a transaction with the wallet
        """
        # Get public key
        public_key = self.public_key.exportKey(format='DER')
        # Use public key as address
        sender_address = binascii.hexlify(public_key).decode('ascii')
        # Build transaction
        trans_dict = {'sender_address': sender_address,
                      'host_address': host_address,
                      'time_slot': time_slot,
                      'amount': amount}
    
        signature = self.sign(trans_dict)
        
        return {'message': trans_dict, 'signature': signature}
    
    
    def show_appointments(self, blockchain):
        """
        Show all booked appointments
        """
        # Get completed transactions
        completed_trans = []
        for block in blockchain.chain:
            for temp in block['transactions']:
                completed_trans.append(temp['message'])
        
        # Get public key
        public_key = self.public_key.exportKey(format='DER')
        # Use public key as address
        address = binascii.hexlify(public_key).decode('ascii')
        
        # Get the appointments
        appointments = []
        for completed_tran in completed_trans:
            if completed_tran['sender_address'] == address:

                # Get booked time slot
                booked_slot = completed_tran['time_slot']
                booked_slot = [time.strip() for time in booked_slot.split('-')]
                
                appointments.append(" - ".join(booked_slot))
                
        return appointments

## Build a Blockchain

In [522]:
class Blockchain:
    """
    Basic blockchain written in implemented in Python.
    """
    
    def __init__(self):
        self.chain = [] # Holds all blocks in the chain
        self.transactions = [] # To store transactions before adding to blockchain
        self.create_block(proof = 1, previous_hash = '0') # Create genesis block
        self.nodes = {} # Hold addresses of nodes
        # Blockchain : [Location, Fee]
        # Blocked requesters
        
    
    def __repr__(self):
        nodeid = id(self)
        return f"Blockchain Node ID: {nodeid}"
    
    
    def create_block(self, proof, previous_hash):
        """
        Create a block and add it to the block chain
        """
        block = {'index': len(self.chain)+1,
                 'timestamp': str(datetime.now()),
                 'proof': proof,
                 'previous_hash': previous_hash,
                 'transactions': self.transactions}
        self.transactions = [] # Empty transactions
        self.chain.append(block) # Append the block to the chain
        return block


    def get_previous_block(self):
        """
        Get the previous block
        """
        return self.chain[-1]


    def proof_of_work(self, previous_proof):
        """
        To find new proof of work (AKA nonce)
        """
        # Define new proof
        new_proof = 1
        check_proof = False
        
        # Loop till prrof is found
        while check_proof is False:
            # Use a non  commutative equation with new and previous proof
            # Find the hash of equation
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            
            # Verify is suitable hash found
            # Increase required number of 0's to increase difficulty
            if hash_operation[:4] == '0000':
                # Found proof, break loop
                check_proof = True
            else:
                # Increment & try again
                new_proof += 1
        
        # Return new proof
        return new_proof


    def hash(self, block):
        """
        Return hash of the block
        """
        # Convert the dict to json
        # Encode to UTF-8
        encoded_block = json.dumps(block, sort_keys = True).encode()
        
        # Hash the encoded block
        # Returns the encoded data in hexadecimal format
        return hashlib.sha256(encoded_block).hexdigest()


    def is_chain_valid(self, chain):
        """
        Check if chain is valid
        """
        # Define previous block and index
        previous_block = chain[0]
        block_index = 1

        # Loop through the blockchain
        while block_index < len(chain):
            # Get block
            block = chain[block_index]

            # Verify previous hash and block
            if block['previous_hash'] != self.hash(previous_block):
                return False

            # Verify previous proof and proof
            previous_proof = previous_block['proof']
            new_proof = block['proof']
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()

            if hash_operation[:4] != '0000':
                return False

            # Increment and continue
            previous_block = block
            block_index += 1

        # No errors found, return valid
        return True


    def validate_transaction(self, transaction):
        """
        Check if transaction is valid
        """
        # Get current transaction
        trans = transaction['message']
        address = trans['sender_address']
        
        # Verify signature
        if self.verify_signature(address, transaction):
            
            # Get completed transactions
            completed_trans = []
            for block in self.chain:
                for temp in block['transactions']:
                    completed_trans.append(temp['message'])
            
            # Get time slot
            if trans['amount'] > 0:
                form = '%d/%m/%y %H:%M:%S'
                time_slot = trans['time_slot']
                start, end = [datetime.strptime(time.strip(), form) 
                              for time in time_slot.split('-')]
    
            
            # Get amount
            amount = trans['amount']
            
            balance = 0
            for completed_tran in completed_trans:
                if completed_tran['host_address'] == trans['host_address']:
                    # Add amount to balance
                    temp = completed_tran['amount']
                    balance += temp if (temp > 0) else (-1)*temp
                    
                    # Get booked time slot
                    if completed_tran['amount'] > 0:
                        booked_slot = completed_tran['time_slot']
                        booked_slot = [datetime.strptime(time.strip(), form) 
                                       for time in booked_slot.split('-')]
                        
                        # Overlapping slot
                        if all([start <= x <= end for x in booked_slot]):
                            # Invalid Transaction
                            return False
            
            # Insufficient balance
            if balance < amount:
                # Invalid Transaction
                return False
            
            # No issue
            return True
        
        # Invalid Signature
        return False
        
        
    def add_transaction(self, transaction):
        """
        Add transaction to the blockchain
        """
        # Validate transaction 
        if self.validate_transaction(transaction):
            
            # Add the transaction
            self.transactions.append(transaction)
            
            # Get previous block
            previous_block = self.get_previous_block()
            # Return index of new block
            return previous_block['index'] + 1
        
        # Invalid Transaction
        return None


    def verify_signature(self, wallet_address, transaction):
        """
        Check that the provided `signature` corresponds to `message`
        signed by the wallet at `wallet_address`
        """
        # Get message and signature from transaction
        message = str(transaction['message'])
        signature = transaction['signature']
        
        # Import public key from address
        pubkey = RSA.importKey(binascii.unhexlify(wallet_address))
        
        # Verify message
        verifier = PKCS1_v1_5.new(pubkey)
        h = SHA.new(message.encode('utf8'))
        return verifier.verify(h, binascii.unhexlify(signature))


    def add_node(self, node):
        """
        Add node to the network
        """
        address = node['blockchain']
        location = node['location']
        fee = node['fee']
        
        # Add to nodes list
        if address not in self.nodes.keys():
            # Address not in nodes yet
            self.nodes[address] = {'location': location,
                                      'fee': fee}


    def replace_chain(self):
        """
        Consensus of our network
        Check if you have the longest chain and replace chain if not
        """
        # Get all nodes
        network = self.nodes.keys()
        longest_chain = None
        
        # Init max length as current length
        max_length = len(self.chain)
        
        # Loop all nodes in the network
        for node in network:
            # Get chain
            response = node.chain
            
            # Get chain and chain length
            length = len(response)
            chain = response

            # Check lengths
            if length > max_length and self.is_chain_valid(chain):
                max_length = length
                longest_chain = chain
        
        # Replace current chain with ongest chain
        if longest_chain:
            self.chain = longest_chain
            return True
        
        # Not replaced
        return False

## Helper Functions

### For Wallets

In [523]:
def generate_wallet(wallet):
    # Get wallet keys
    public_key = wallet.public_key.exportKey(format = 'DER')
    private_key = wallet.private_key.exportKey(format = 'DER')
    
    # Decode them to a readable form
    public_key = binascii.hexlify(public_key).decode('ascii')
    private_key = binascii.hexlify(private_key).decode('ascii')
    
    response = {'Public Key': public_key,
                'Private Key': private_key,
                'Wallet Address': public_key}
    
    return response


def new_appointment(data, wallet, blockchain):    
    # Transaction keys
    transaction_keys = ['host_address', 'time_slot','amount']
        
    # Check if transaction has all keys
    if not all (key in data for key in transaction_keys):
    	# Return response code
        return "Missing key", 400
    
    # Create transaction
    transaction = wallet.create_transaction(data['host_address'],
                                            data['time_slot'],
                                            data['amount'])
    
    # Should be broadcast to all nodes
    # Loop over the network
    for node in blockchain.nodes:
        # Post the transaction to the node
        node.add_transaction(transaction)

    # Create a response
    response = {'Message': 'New transaction created'}
    return response


def show_appointments(wallet, blockchain):
    # Get booked appointments
    appointments = wallet.show_appointments(blockchain)
    
    # Create a response
    response = {'Message': 'Showing booked appointments',
                'Appointments': appointments}
    return response

### For Blockchain

In [524]:
def mine_block(wallet, blockchain):
    # Get previous block and proof
    previous_block = blockchain.get_previous_block()
    previous_proof = previous_block['proof']

    # Calculate proof of work
    proof = blockchain.proof_of_work(previous_proof)
    
    # Calculate previous hash
    previous_hash = blockchain.hash(previous_block)
    
    # Add the mining reward
    temp = wallet.create_transaction(wallet.address, "00:00", -1)
    
    # Add transaction to the blockchain
    blockchain.add_transaction(temp)
    
    # Add the block to the blockchain
    block = blockchain.create_block(proof, previous_hash)

    # Create response to send
    response = {'Message': 'Congrats! You just mined a block.',
                'index': block['index'],
                'timestamp': block['timestamp'],
                'proof': block['proof'],
                'previous_hash': block['previous_hash'],
                'transactions': block['transactions']}

    return response


def get_chain(blockchain):
    # Get chain
    response = {'chain': blockchain.chain,
                'length': len(blockchain.chain)}

    return response


def chain_valid(blockchain):
    # Verify the blockchain
    is_valid = blockchain.is_chain_valid(blockchain.chain)

    # Create response
    if is_valid:
        response = {'Message': 'Yes'}
    else:
        response = {'Message': 'No'}
        
    return response


def add_transaction(data, blockchain):
    # Transaction keys
    transaction_keys = ['transaction']
        
    # Check if transaction has all keys
    if not all (key in data for key in transaction_keys):
    	# Return response code
        return "Missing key", 400
    
    # Add transaction to the blockchain
    index = blockchain.add_transactions(data['transaction'])

    # Create a response
    response = {'Message': 'This transactions will be added', 'Index': index}
    return response


def connect_nodes(data, blockchain):
    # Nodes currently in the network
    nodes = data["nodes"]

    # If there are no nodes in the network
    if nodes is None:
        return "No nodes", 400

    # Loop over all nodes in the network
    for node in nodes:
        # Send new node address to all nodes
        blockchain.add_node(node)

    # Create response
    response = {'Message': 'All nodes connected. The nodes are: ',
                'total_nodes': list(blockchain.nodes)}

    return response


def replace_chain(blockchain):
    # Replace the chain
    is_chain_replaced = blockchain.replace_chain()
    
    # If chain has been replaced
    # Create response
    if is_chain_replaced:
        response = {'message': 'The nodes had different chains so the chain was replaced by the longest one.',
                    'new_chain': blockchain.chain}
    else:
        response = {'message': 'All good. The chain is the largest one.',
                    'actual_chain': blockchain.chain}
    return response


def show_trans(blockchain):
    # Show all transactions
    response = {'message': 'Showing all unconfirmed transactions',
                'transactions': blockchain.transactions}
    
    return response


def show_hosts(blockchain):
    # Get nodes
    nodes = blockchain.nodes
    locations = [node[0] for node in nodes if node[0] != "0,0"]
    
    # Create a response
    response = {'Message': 'Showing all host locations',
                'Locations': locations}
    return response

## Create Nodes

### Node 1

In [525]:
b1 = Blockchain()
w1 = Wallet()

### Node 2

In [526]:
b2 = Blockchain()
w2 = Wallet()

### Node 3

In [527]:
b3 = Blockchain()
w3 = Wallet()

#### Consider Node 1 and Node 2 as full nodes (miners) and Node 3 as a client

## Connect all nodes

In [528]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [529]:
nodes = {"nodes": [{"blockchain": b1, "location": "4,2", "fee": 0.25},
                   {"blockchain": b2, "location": "3,1", "fee": 0.2},
                   {"blockchain": b3, "location": "0,0", "fee": 0}]}

In [530]:
response = connect_nodes(nodes, b1)
pp.pprint(response)

{   'Message': 'All nodes connected. The nodes are: ',
    'total_nodes': [   Blockchain Node ID: 140105931038528,
                       Blockchain Node ID: 140105929855760,
                       Blockchain Node ID: 140105931674144]}


In [531]:
response = connect_nodes(nodes, b2)
pp.pprint(response)

{   'Message': 'All nodes connected. The nodes are: ',
    'total_nodes': [   Blockchain Node ID: 140105931038528,
                       Blockchain Node ID: 140105929855760,
                       Blockchain Node ID: 140105931674144]}


In [532]:
response = connect_nodes(nodes, b3)
pp.pprint(response)

{   'Message': 'All nodes connected. The nodes are: ',
    'total_nodes': [   Blockchain Node ID: 140105931038528,
                       Blockchain Node ID: 140105929855760,
                       Blockchain Node ID: 140105931674144]}


## Generate Wallet

In [533]:
response = generate_wallet(w3)
pp.pprint(response)

{   'Private Key': '3082025d02010002818100888fc7ec7413a13a1ccafd209e96980b58cfc45dd9f328fa0da2f4a9f666cc1dfa95e2f98579210674b11569709f7274a743b9fea861955640eb7afc1baf7b10f4a03e75038039ba57a33ebb27e003890cb983c8e6d7f120799a3ce03393acf8ec391dc8ebe6a9459ce7c8468810152f8898330eaa14a4acc494a425279684f902030100010281805388e839aad3ab271a1fcd715bb130f697f017e57501fa49e37790043bb6ac1b968283623114647047875f5aad4acbbfb1e58aebcce24c7083abc80259892154dc1dfa3271bee4568b957dda81dcdf4d63c1aae143db9b8dcd2eb9e487b1859d19e322dd8ea5388bb674880321db63d21749df6f596eef98fa5b8edc8bdc54d1024100b9c6d7e94ac47ae941486ea055e59ce86a71555145874841234e2d8686c207499aa219ac3a6fad22586ebdb7b894994bfb5cec2186128b453c97343337e6a66d024100bc2e83c89c49f67a9941c094b6ca98863db63605aa2279988535cc71d91277a890d3aec2f4f1e277a152176054424009c51b826f2173ba520e5b3c9d8b1e313d024100b8b315d48e8e3b790e7bb9e8d49d9d5b81003e87b7ada14b89935f35c162c4a60c041fac18c91103f591dc32fd6357e9aff61999d6d736f9d61caa3c45d69e1d024100828b623090fb5e59be67b6

## Create Chains

In [534]:
response = get_chain(b1)
pp.pprint(response)

{   'chain': [   {   'index': 1,
                     'previous_hash': '0',
                     'proof': 1,
                     'timestamp': '2020-06-09 23:24:55.365240',
                     'transactions': []}],
    'length': 1}


In [535]:
response = get_chain(b2)
pp.pprint(response)

{   'chain': [   {   'index': 1,
                     'previous_hash': '0',
                     'proof': 1,
                     'timestamp': '2020-06-09 23:24:55.757112',
                     'transactions': []}],
    'length': 1}


In [536]:
response = get_chain(b3)
pp.pprint(response)

{   'chain': [   {   'index': 1,
                     'previous_hash': '0',
                     'proof': 1,
                     'timestamp': '2020-06-09 23:24:56.060332',
                     'transactions': []}],
    'length': 1}


## Mine a Block

In [537]:
response = mine_block(w1, b1)
pp.pprint(response)

{   'Message': 'Congrats! You just mined a block.',
    'index': 2,
    'previous_hash': '4badd8f8d185e27b144ca3edd59c4702a5a4f884a49c59962f3a4d140ce08074',
    'proof': 533,
    'timestamp': '2020-06-09 23:24:58.228445',
    'transactions': [   {   'message': {   'amount': -1,
                                           'host_address': '30819f300d06092a864886f70d010101050003818d00308189028181009b88a7bc94eb3c268bdef83b04bfd4b866a332de46a692eddcd12958e24a5ed29b7c18254686a3193668be69cea616361203efe61a3356f545c417c694ccc875c0d7eb174c86be0f23940c21cd707b451152249becbdbd9875a58e98e375ac0ed738254ba6c2a03d780fe38b457f82eefecae230b161618e04ba32f811de07150203010001',
                                           'sender_address': '30819f300d06092a864886f70d010101050003818d00308189028181009b88a7bc94eb3c268bdef83b04bfd4b866a332de46a692eddcd12958e24a5ed29b7c18254686a3193668be69cea616361203efe61a3356f545c417c694ccc875c0d7eb174c86be0f23940c21cd707b451152249becbdbd9875a58e98e375ac0ed738254ba6c2a03d780fe3

## Consensus/Replace Chain

In [538]:
response = replace_chain(b1)
pp.pprint(response)

{   'actual_chain': [   {   'index': 1,
                            'previous_hash': '0',
                            'proof': 1,
                            'timestamp': '2020-06-09 23:24:55.365240',
                            'transactions': []},
                        {   'index': 2,
                            'previous_hash': '4badd8f8d185e27b144ca3edd59c4702a5a4f884a49c59962f3a4d140ce08074',
                            'proof': 533,
                            'timestamp': '2020-06-09 23:24:58.228445',
                            'transactions': [   {   'message': {   'amount': -1,
                                                                   'host_address': '30819f300d06092a864886f70d010101050003818d00308189028181009b88a7bc94eb3c268bdef83b04bfd4b866a332de46a692eddcd12958e24a5ed29b7c18254686a3193668be69cea616361203efe61a3356f545c417c694ccc875c0d7eb174c86be0f23940c21cd707b451152249becbdbd9875a58e98e375ac0ed738254ba6c2a03d780fe38b457f82eefecae230b161618e04ba32f811de071502030

In [539]:
response = replace_chain(b2)
pp.pprint(response)

{   'message': 'The nodes had different chains so the chain was replaced by '
               'the longest one.',
    'new_chain': [   {   'index': 1,
                         'previous_hash': '0',
                         'proof': 1,
                         'timestamp': '2020-06-09 23:24:55.365240',
                         'transactions': []},
                     {   'index': 2,
                         'previous_hash': '4badd8f8d185e27b144ca3edd59c4702a5a4f884a49c59962f3a4d140ce08074',
                         'proof': 533,
                         'timestamp': '2020-06-09 23:24:58.228445',
                         'transactions': [   {   'message': {   'amount': -1,
                                                                'host_address': '30819f300d06092a864886f70d010101050003818d00308189028181009b88a7bc94eb3c268bdef83b04bfd4b866a332de46a692eddcd12958e24a5ed29b7c18254686a3193668be69cea616361203efe61a3356f545c417c694ccc875c0d7eb174c86be0f23940c21cd707b451152249becbdbd9875a58

In [540]:
response = replace_chain(b3)
pp.pprint(response)

{   'message': 'The nodes had different chains so the chain was replaced by '
               'the longest one.',
    'new_chain': [   {   'index': 1,
                         'previous_hash': '0',
                         'proof': 1,
                         'timestamp': '2020-06-09 23:24:55.365240',
                         'transactions': []},
                     {   'index': 2,
                         'previous_hash': '4badd8f8d185e27b144ca3edd59c4702a5a4f884a49c59962f3a4d140ce08074',
                         'proof': 533,
                         'timestamp': '2020-06-09 23:24:58.228445',
                         'transactions': [   {   'message': {   'amount': -1,
                                                                'host_address': '30819f300d06092a864886f70d010101050003818d00308189028181009b88a7bc94eb3c268bdef83b04bfd4b866a332de46a692eddcd12958e24a5ed29b7c18254686a3193668be69cea616361203efe61a3356f545c417c694ccc875c0d7eb174c86be0f23940c21cd707b451152249becbdbd9875a58

## Validate Chain

In [541]:
response = chain_valid(b1)
pp.pprint(response)

{'Message': 'Yes'}


In [542]:
response = chain_valid(b2)
pp.pprint(response)

{'Message': 'Yes'}


In [543]:
response = chain_valid(b3)
pp.pprint(response)

{'Message': 'Yes'}


## Create New Appointment

In [544]:
data = {"host_address": w1.address,
        "time_slot": "10/06/20 16:30:00 - 10/06/20 17:00:00",
        "amount": 0.5}

In [545]:
response = new_appointment(data, w3, b3)
pp.pprint(response)

{'Message': 'New transaction created'}


## Show all Unconfirmed Transaction

In [546]:
response = show_trans(b1)
pp.pprint(response)

{   'message': 'Showing all unconfirmed transactions',
    'transactions': [   {   'message': {   'amount': 0.5,
                                           'host_address': '30819f300d06092a864886f70d010101050003818d00308189028181009b88a7bc94eb3c268bdef83b04bfd4b866a332de46a692eddcd12958e24a5ed29b7c18254686a3193668be69cea616361203efe61a3356f545c417c694ccc875c0d7eb174c86be0f23940c21cd707b451152249becbdbd9875a58e98e375ac0ed738254ba6c2a03d780fe38b457f82eefecae230b161618e04ba32f811de07150203010001',
                                           'sender_address': '30819f300d06092a864886f70d010101050003818d0030818902818100888fc7ec7413a13a1ccafd209e96980b58cfc45dd9f328fa0da2f4a9f666cc1dfa95e2f98579210674b11569709f7274a743b9fea861955640eb7afc1baf7b10f4a03e75038039ba57a33ebb27e003890cb983c8e6d7f120799a3ce03393acf8ec391dc8ebe6a9459ce7c8468810152f8898330eaa14a4acc494a425279684f90203010001',
                                           'time_slot': '10/06/20 16:30:00 - '
                                

In [547]:
response = show_trans(b2)
pp.pprint(response)

{   'message': 'Showing all unconfirmed transactions',
    'transactions': [   {   'message': {   'amount': 0.5,
                                           'host_address': '30819f300d06092a864886f70d010101050003818d00308189028181009b88a7bc94eb3c268bdef83b04bfd4b866a332de46a692eddcd12958e24a5ed29b7c18254686a3193668be69cea616361203efe61a3356f545c417c694ccc875c0d7eb174c86be0f23940c21cd707b451152249becbdbd9875a58e98e375ac0ed738254ba6c2a03d780fe38b457f82eefecae230b161618e04ba32f811de07150203010001',
                                           'sender_address': '30819f300d06092a864886f70d010101050003818d0030818902818100888fc7ec7413a13a1ccafd209e96980b58cfc45dd9f328fa0da2f4a9f666cc1dfa95e2f98579210674b11569709f7274a743b9fea861955640eb7afc1baf7b10f4a03e75038039ba57a33ebb27e003890cb983c8e6d7f120799a3ce03393acf8ec391dc8ebe6a9459ce7c8468810152f8898330eaa14a4acc494a425279684f90203010001',
                                           'time_slot': '10/06/20 16:30:00 - '
                                

In [548]:
response = show_trans(b3)
pp.pprint(response)

{   'message': 'Showing all unconfirmed transactions',
    'transactions': [   {   'message': {   'amount': 0.5,
                                           'host_address': '30819f300d06092a864886f70d010101050003818d00308189028181009b88a7bc94eb3c268bdef83b04bfd4b866a332de46a692eddcd12958e24a5ed29b7c18254686a3193668be69cea616361203efe61a3356f545c417c694ccc875c0d7eb174c86be0f23940c21cd707b451152249becbdbd9875a58e98e375ac0ed738254ba6c2a03d780fe38b457f82eefecae230b161618e04ba32f811de07150203010001',
                                           'sender_address': '30819f300d06092a864886f70d010101050003818d0030818902818100888fc7ec7413a13a1ccafd209e96980b58cfc45dd9f328fa0da2f4a9f666cc1dfa95e2f98579210674b11569709f7274a743b9fea861955640eb7afc1baf7b10f4a03e75038039ba57a33ebb27e003890cb983c8e6d7f120799a3ce03393acf8ec391dc8ebe6a9459ce7c8468810152f8898330eaa14a4acc494a425279684f90203010001',
                                           'time_slot': '10/06/20 16:30:00 - '
                                

## Mine New Block and Update Chain

In [549]:
response = mine_block(w2, b2)
pp.pprint(response)

{   'Message': 'Congrats! You just mined a block.',
    'index': 3,
    'previous_hash': '7b681966c8adf8ed14c27298dd6cc47783ba0b7d92d07e8ac726fab030555978',
    'proof': 45293,
    'timestamp': '2020-06-09 23:25:00.451930',
    'transactions': [   {   'message': {   'amount': 0.5,
                                           'host_address': '30819f300d06092a864886f70d010101050003818d00308189028181009b88a7bc94eb3c268bdef83b04bfd4b866a332de46a692eddcd12958e24a5ed29b7c18254686a3193668be69cea616361203efe61a3356f545c417c694ccc875c0d7eb174c86be0f23940c21cd707b451152249becbdbd9875a58e98e375ac0ed738254ba6c2a03d780fe38b457f82eefecae230b161618e04ba32f811de07150203010001',
                                           'sender_address': '30819f300d06092a864886f70d010101050003818d0030818902818100888fc7ec7413a13a1ccafd209e96980b58cfc45dd9f328fa0da2f4a9f666cc1dfa95e2f98579210674b11569709f7274a743b9fea861955640eb7afc1baf7b10f4a03e75038039ba57a33ebb27e003890cb983c8e6d7f120799a3ce03393acf8ec391dc8ebe6a9459ce

In [550]:
response = replace_chain(b1)
response = replace_chain(b2)
response = replace_chain(b3)

## Show Booked Appointments

In [551]:
response = show_appointments(w3,b3)
pp.pprint(response)

{   'Appointments': ['10/06/20 16:30:00 - 10/06/20 17:00:00'],
    'Message': 'Showing booked appointments'}
